In [1]:
import os
import numpy as np
import faiss
import ollama
from tqdm import tqdm
import pickle

In [ ]:


# 步骤2: 配置常量
EMBEDDING_MODEL = "nomic-embed-text"
VECTOR_STORE_DIR = "./vector_stores"
INDEX_NAME = "my_documents"
MODEL_NAME = "llama3.2:latest" 

# 步骤3: 加载向量存储
def load_vector_store(index_name):
    index_path = os.path.join(VECTOR_STORE_DIR, f"{index_name}.faiss")
    metadata_path = os.path.join(VECTOR_STORE_DIR, f"{index_name}_metadata.pkl")
    
    try:
        index = faiss.read_index(index_path)
        with open(metadata_path, "rb") as f:
            data = pickle.load(f)
            chunks = data['chunks']
            metadata = data['metadata']
        print(f"✅ 成功加载向量存储 | 索引大小: {index.ntotal}")
        return index, chunks, metadata
    except Exception as e:
        print(f"❌ 加载失败: {str(e)}")
        return None, None, None

# 加载向量存储
index, chunks, metadata = load_vector_store(INDEX_NAME)

# 步骤4: 定义查询函数
def query_documents(question, index, chunks, metadata, model_name=MODEL_NAME, k=3):
    """查询文档并获取回答"""
    try:
        # 获取问题嵌入
        response = ollama.embeddings(model=EMBEDDING_MODEL, prompt=question)
        query_embedding = np.array([response['embedding']], dtype=np.float32)
        
        # 搜索相似内容
        distances, indices = index.search(query_embedding, k)
        
        # 获取相关文本块
        context_chunks = [chunks[i] for i in indices[0]]
        context_metadata = [metadata[i] for i in indices[0]]
        
        # 构造上下文
        context = "\n\n".join([
            f"来源: {meta['filename']} 第{meta['page']}页\n内容: {chunk}"
            for chunk, meta in zip(context_chunks, context_metadata)
        ])
        
        # 构造提示
        prompt = f"""
        基于以下上下文信息回答问题：
        {context}
        
        问题: {question}
        回答:
        """
        
        # 获取回答
        response = ollama.chat(
            model=model_name,
            messages=[{'role': 'user', 'content': prompt}]
        )
        
        return response['message']['content'], context_metadata
        
    except Exception as e:
        return f"查询失败: {str(e)}", []

# 步骤5: 交互式文档查询
print("🚀 文档助手已启动! 输入 'exit' 退出")

✅ 成功加载向量存储 | 索引大小: 60
🚀 文档助手已启动! 输入 'exit' 退出


In [3]:
# 直接在单元格中提问
question = "what is energy distance"
answer, sources = query_documents(question, index, chunks, metadata)

print(f"🤖 助手回答:\n{answer}\n")
print("📚 来源:")
for i, source in enumerate(sources):
    print(f"{i+1}. {source['filename']} - 第{source['page']}页")

🤖 助手回答:
根据给出的文本，Energy Distance 是一种统计量，是一种基于距离的统计量，它描述了两个多维随机变量X和Y之间的关系。

具体来说，Energy Distance 定义为：

E(X,Y) = 2E[ Xi - Yj ] + E[X' - Xj] + E[Y' - Yj]

其中，Xi 和 Yi 是 independent 的一维 random variable，X' 和 Y' 是 iid 复制的 random variable。

此定义可以通过 Parseval-Plancherel 公式来推导出来，这个公式描述了多维空间中函数之间的内积与二维空间中函数之间的内积之间的关系。

根据 Székely 和 Rizzo (2005a) 的研究，Energy Distance 是一种非负的统计量，它可以用来衡量两个 random variable 之间的相似性或差异。

总之，Energy Distance 是一种基于距离的统计量，描述了两个多维随机变量之间的关系和相似性。

📚 来源:
1. Energy statistics- A class of statistics based on distances.pdf - 第4页
2. Energy statistics- A class of statistics based on distances.pdf - 第3页
3. Energy statistics- A class of statistics based on distances.pdf - 第5页
